# Differential analysis of abundances in omnics dataset: hypothesis testing, multiple comparisons problem and corrections.

## Purpose:

Here, we show the user how to perform hypothesis testing and how to determine fold-change of expressions to extract out differentially expressed elements in their omnics datasets. In addition, we will briefly introduce the problems of multiple comparisons and the methods for eliminating type I errors in their analyses. 

First, we need to import the neccessary Python packages for this module.

In [48]:
import pandas as pd
import numpy as np
import scipy.stats as sts
from statsmodels.stats import multitest

## Trial 1: 

There will be three comparison that we will focus on in this dataset:

$$\Delta cheA4\ versus\ WT\ (SP7)$$
$$\Delta cheA1\Delta cheA4\ versus\ WT\ (SP7)$$
$$\Delta cheA1\Delta cheA4\ versus\ \Delta cheA4$$

First, we import the cleaned dataset from **trial 1** from the *pre-processing* step.

In [49]:
data_cleaned = pd.read_excel('cleaned_trial1.xlsx')
data_cleaned.head()

,Unnamed: 0,Accession,Gene function,"Abundances (Normalized): F7: Sample, Bio Rep1, SP7","Abundances (Normalized): F8: Sample, Bio Rep2, SP7","Abundances (Normalized): F9: Sample, Bio Rep3, SP7","Abundances (Normalized): F4: Sample, Bio Rep1, CheA4","Abundances (Normalized): F5: Sample, Bio Rep2, CheA4","Abundances (Normalized): F6: Sample, Bio Rep3, CheA4","Abundances (Normalized): F1: Sample, Bio Rep1, CheA1CheA4","Abundances (Normalized): F2: Sample, Bio Rep2, CheA1CheA4","Abundances (Normalized): F3: Sample, Bio Rep3, CheA1CheA4"
0,0,A0A0P0FBD5,Uncharacterized protein,4.199953e+08,5.327991e+08,1.449827e+08,6.119378e+07,1.042649e+08,1.139954e+08,2.483043e+07,2.529656e+07,3.566432e+07
1,1,A0A0P0F6W5,Uncharacterized protein,1.155383e+07,6.643362e+06,1.302690e+07,3.419722e+07,3.787835e+07,3.694340e+07,1.599574e+07,1.121537e+07,2.037844e+07
2,2,A0A0N7I7H6,Uncharacterized protein,2.349336e+09,2.385109e+09,2.635075e+09,6.572775e+08,6.953822e+08,6.456934e+08,2.195988e+09,2.295658e+09,2.148146e+09
3,3,A0A0P0ENT2,Uncharacterized protein,1.385459e+08,2.930326e+08,3.267244e+08,2.197483e+07,2.720034e+07,1.437205e+07,1.708144e+08,2.937530e+08,1.056491e+08
4,4,A0A0P0EGU7,Uncharacterized protein,7.191629e+09,7.548888e+09,2.844535e+09,2.296458e+08,2.342704e+08,2.118544e+08,1.384437e+10,1.546637e+10,2.970876e+09


## Performing pairwise statistical analysis to determine differences of protein abundances between two samples.

From here, we want to perform pairwise comparision between two mutants at a time. We will be using **Welch's T-test** to perform comparision of mean between two different sample. This method assume unequal variance. 

Though our datasets has been normalized to a mean and standard deviation, performing the Student's T-test with unequal variances condition can take into account any outliers of protein abundances that can skew the distrubution of each sample. Plus, the Welch's t-test is more robust compared to Student's t-test and will give similar statistical results compared to the Student's t-test if both samples of interest have similar variances.

For more information, please refer to:
> Ruxton, G. D. (2006). "The unequal variance t-test is an underused alternative to Student's t-test and the Mann–Whitney U test". Behavioral Ecology. 17: 688–690. doi:10.1093/beheco/ark016.

In [50]:
#Setting index for each mutants for downstream statistical analysis
index = data_cleaned.columns
index_sp7 = index[3:6]
index_A4 = index[6:9]
index_A1A4 = index[9:12]

In [51]:
# Using the toolbox from the scipy.stats package, we will perform the two-sided Welch's t-test between 
# pairwise comparisions.

p_values_A4sp7 = []
for i in range(0, len(data_cleaned)):
    sp7 = data_cleaned.loc[i][index_sp7].values
    A4 = data_cleaned.loc[i][index_A4].values
    p_values_A4sp7.append(sts.ttest_ind(A4, sp7, equal_var=False).pvalue)

p_values_A1A4sp7 = []
for i in range(0, len(data_cleaned)):
    sp7 = data_cleaned.loc[i][index_sp7].values
    A1A4 = data_cleaned.loc[i][index_A1A4].values
    p_values_A1A4sp7.append(sts.ttest_ind(A1A4, sp7, equal_var=False).pvalue)

p_values_A4A1A4 = []
for i in range(0, len(data_cleaned)):
    A4 = data_cleaned.loc[i][index_A4].values
    A1A4 = data_cleaned.loc[i][index_A1A4].values
    p_values_A4A1A4.append(sts.ttest_ind(A1A4, A4, equal_var=False).pvalue)

## Accounting for the problems with multiple comparisons testing.

For the introduction of problems associated with multiple comparison testing, please refer to:
> https://en.wikipedia.org/wiki/Multiple_comparisons_problem#Large-scale_multiple_testing

In brief, we are simultaneously performing hypotheses testings , over different conditions, for more than thoudsands of proteins within our big biological dataset **with no prior expectation or basis of what specific proteins are expected to change**. We are **infering, based on our testing parameters**,  the differentially expressed elements based on our observations of protein abundances. Because of this, we are expecting that some of the differentially expressed elements to be **false positives**. To account for this error, we must perform a correction of our p-values to account for this problem in multiple comparisons and reduce type I error.

To do this, we are performing a correction method called **bonferroni correction**. If you want to read more on this method, please refer to:
> https://www.ncbi.nlm.nih.gov/pubmed/24697967

Traditionally, we expect that the **false discovery rate** to be 0.05, meaning that from the pool of differentially expressed elements, we expect about 5% of those elements to be **false positive**. 

In [52]:
# From the statsmodels.stats package, we can perform the Bonferroni correction method using the 
# multitest module.

qvs_A4vssp7 = multitest.multipletests(p_values_A4sp7 ,method='fdr_bh', alpha=0.05)
qvs_A1A4vssp7 = multitest.multipletests(p_values_A1A4sp7 ,method='fdr_bh', alpha=0.05)
qvs_A1A4vsA4 = multitest.multipletests(p_values_A4A1A4 ,method='fdr_bh', alpha=0.05)

data_cleaned['p_correct_A4vssp7'] = qvs_A4vssp7[1]
data_cleaned['p_correct_A1A4vssp7'] = qvs_A1A4vssp7[1]
data_cleaned['p_correct_A1A4vsA4'] = qvs_A1A4vsA4[1]

#Now, we have to separate the proteins that has p<0.05 for at least one comparison study
data_DEPs = data_cleaned[(data_cleaned['p_correct_A4vssp7'] < 0.05) | (data_cleaned['p_correct_A1A4vssp7'] < 0.05) | (data_cleaned['p_correct_A1A4vsA4'] < 0.05)]
data_DEPs.head()

,Unnamed: 0,Accession,Gene function,"Abundances (Normalized): F7: Sample, Bio Rep1, SP7","Abundances (Normalized): F8: Sample, Bio Rep2, SP7","Abundances (Normalized): F9: Sample, Bio Rep3, SP7","Abundances (Normalized): F4: Sample, Bio Rep1, CheA4","Abundances (Normalized): F5: Sample, Bio Rep2, CheA4","Abundances (Normalized): F6: Sample, Bio Rep3, CheA4","Abundances (Normalized): F1: Sample, Bio Rep1, CheA1CheA4","Abundances (Normalized): F2: Sample, Bio Rep2, CheA1CheA4","Abundances (Normalized): F3: Sample, Bio Rep3, CheA1CheA4",p_correct_A4vssp7,p_correct_A1A4vssp7,p_correct_A1A4vsA4
1,1,A0A0P0F6W5,Uncharacterized protein,1.155383e+07,6.643362e+06,1.302690e+07,34197220.0,3.787835e+07,3.694340e+07,1.599574e+07,1.121537e+07,2.037844e+07,0.005841,0.303602,0.023782
2,2,A0A0N7I7H6,Uncharacterized protein,2.349336e+09,2.385109e+09,2.635075e+09,657277462.9,6.953822e+08,6.456934e+08,2.195988e+09,2.295658e+09,2.148146e+09,0.009008,0.213539,0.003390
6,6,A0A0P0FD78,Peptide ABC transporter substrate-binding protein,3.852770e+08,3.888840e+08,1.777021e+08,600323064.0,4.946053e+08,4.881100e+08,1.629101e+08,1.379317e+08,1.871210e+08,0.101209,0.276157,0.016554
7,7,A0A0P0EW12,DNA helicase,2.759189e+09,2.906706e+09,2.672235e+09,222301899.3,2.533530e+08,1.932872e+08,2.600452e+09,3.042813e+09,7.640314e+08,0.003224,0.553611,0.154110
8,8,A0A0P0F5R5,Glutathione S-transferase,2.089823e+09,2.056875e+09,4.708423e+09,655938818.3,7.985197e+08,7.800253e+08,9.891769e+09,8.797010e+09,1.174020e+10,0.153216,0.047262,0.023764


## Determining the average fold-changes in protein abundances for all comparisons and filtering out differentially expressed proteins

Now that we have determined the corrected p-values from the **Welch's T-test** and **Bonferroni correction**, we can follow up by determining the average fold-changes in protein abundances. 

Traditionally, a differentially expressed element is defined as having: *a significantly different abundance's average compared to the reference* **and** *an average fold-change of at least twice or half compared to the reference*.

We usually represent fold-changes of elements in logrithm of 2. To determine the average fold-change between samples, we can use the following expression:

$$log_2(FC)\ = log_2(Sample_{avg}\div\ Reference_{avg}) = log_2(Sample_{avg}) - log_2(Reference_{avg})$$

In [62]:
mean_sp7 = np.mean(data_DEPs[index_sp7], axis=1)
mean_A4 = np.mean(data_DEPs[index_A4], axis=1)
mean_A1A4 = np.mean(data_DEPs[index_A1A4], axis=1)

data_DEPs['log2_A4/sp7'] = np.log2(mean_A4/mean_sp7)
data_DEPs['log2_A1A4/sp7'] =np.log2(mean_A1A4/mean_sp7)
data_DEPs['log2_A1A4/A4'] =np.log2(mean_A1A4/mean_A4)

Now that we have input the average fold-change values. We can filter out our differentially expressed proteins.

Below is our differentially expressed protein dataset for **trial 1**.

In [54]:
data_DEPs = data_DEPs[(abs(data_DEPs['log2_A4/sp7']) > 1) | (abs(data_DEPs['log2_A1A4/sp7']) > 1) | (abs(data_DEPs['log2_A1A4/A4']) > 1)]
data_DEPs = data_DEPs.reset_index(drop=True)[data_DEPs.columns[1:len(data_DEPs.columns)]]
data_DEPs.head()

,Accession,Gene function,"Abundances (Normalized): F7: Sample, Bio Rep1, SP7","Abundances (Normalized): F8: Sample, Bio Rep2, SP7","Abundances (Normalized): F9: Sample, Bio Rep3, SP7","Abundances (Normalized): F4: Sample, Bio Rep1, CheA4","Abundances (Normalized): F5: Sample, Bio Rep2, CheA4","Abundances (Normalized): F6: Sample, Bio Rep3, CheA4","Abundances (Normalized): F1: Sample, Bio Rep1, CheA1CheA4","Abundances (Normalized): F2: Sample, Bio Rep2, CheA1CheA4","Abundances (Normalized): F3: Sample, Bio Rep3, CheA1CheA4",p_correct_A4vssp7,p_correct_A1A4vssp7,p_correct_A1A4vsA4,log2_A4/sp7,log2_A1A4/sp7,log2_A1A4/A4
0,A0A0P0F6W5,Uncharacterized protein,1.155383e+07,6.643362e+06,1.302690e+07,34197220.0,3.787835e+07,3.694340e+07,1.599574e+07,1.121537e+07,2.037844e+07,0.005841,0.303602,0.023782,1.803848,0.607985,-1.195863
1,A0A0N7I7H6,Uncharacterized protein,2.349336e+09,2.385109e+09,2.635075e+09,657277462.9,6.953822e+08,6.456934e+08,2.195988e+09,2.295658e+09,2.148146e+09,0.009008,0.213539,0.003390,-1.882759,-0.150433,1.732327
2,A0A0P0FD78,Peptide ABC transporter substrate-binding protein,3.852770e+08,3.888840e+08,1.777021e+08,600323064.0,4.946053e+08,4.881100e+08,1.629101e+08,1.379317e+08,1.871210e+08,0.101209,0.276157,0.016554,0.733870,-0.963983,-1.697853
3,A0A0P0EW12,DNA helicase,2.759189e+09,2.906706e+09,2.672235e+09,222301899.3,2.533530e+08,1.932872e+08,2.600452e+09,3.042813e+09,7.640314e+08,0.003224,0.553611,0.154110,-3.639771,-0.380008,3.259763
4,A0A0P0F5R5,Glutathione S-transferase,2.089823e+09,2.056875e+09,4.708423e+09,655938818.3,7.985197e+08,7.800253e+08,9.891769e+09,8.797010e+09,1.174020e+10,0.153216,0.047262,0.023764,-1.986570,1.780862,3.767432


## Trial 2: 

We will repeat the process with this dataset. There will be three comparison that we will focus on in this dataset:

$$\Delta cheA1\ versus\ WT\ (SP7)$$
$$\Delta cheA1(pBBRTMX) versus\ WT\ (SP7)$$
$$\Delta cheA1\ versus\ \Delta cheA1(pBBRTMX)$$

First, we import the cleaned dataset from **trial 2** from the *pre-processing* step.

In [55]:
data_cleaned = pd.read_excel('cleaned_trial2.xlsx')
data_cleaned.head()

,Unnamed: 0,CheA1_pBBR_TMX_Rep01,CheA1_pBBR_TMX_Rep02,CheA1_pBBR_TMX_Rep03,CheA1_Rep01,CheA1_Rep02,CheA1_Rep03,sp7_Rep01,sp7_Rep02,sp7_Rep03,Accession,Gene function
0,0,27.678230,28.351285,28.118906,27.011518,27.564398,28.045627,27.414860,27.214066,27.412692,A0A060D8U1,Cold-shock protein (Putative cold-shock DNA-bi...
1,1,29.920970,28.593295,30.462980,30.902809,30.582126,30.702943,30.896710,30.706459,31.226559,A0A060D8W4,50S ribosomal protein L35
2,2,28.021528,26.481954,26.949522,26.466264,28.467305,25.593041,27.428749,27.791901,27.853953,A0A060D9F9,Chemotaxis protein CheY (DNA-binding response ...
3,3,28.515878,27.860462,27.642605,26.758032,26.813398,28.224904,28.262294,26.007408,28.672895,A0A060D9K4,LysR family transcriptional regulator
4,4,28.257412,28.057875,28.354392,28.512585,28.649630,28.435946,27.901150,27.756669,28.366099,A0A060D9N9,Nucleoid-associated protein ABAZ39_01980


In [56]:
#Setting index for each mutants for downstream statistical analysis
index = data_cleaned.columns
index_sp7 = index[7:10]
index_A1 = index[4:7]
index_A1pbbrTMX = index[1:4]

In [57]:
# Using the toolbox from the scipy.stats package, we will perform the two-sided Welch's t-test between 
# pairwise comparisions.

p_values_A1sp7 = []
for i in range(0, len(data_cleaned)):
    sp7 = data_cleaned.loc[i][index_sp7].values
    A1 = data_cleaned.loc[i][index_A1].values
    p_values_A1sp7.append(sts.ttest_ind(A1, sp7, equal_var=False).pvalue)

p_values_A1TMXsp7 = []
for i in range(0, len(data_cleaned)):
    sp7 = data_cleaned.loc[i][index_sp7].values
    A1TMX = data_cleaned.loc[i][index_A1pbbrTMX].values
    p_values_A1TMXsp7.append(sts.ttest_ind(A1TMX, sp7, equal_var=False).pvalue)

p_values_A1A1TMX = []
for i in range(0, len(data_cleaned)):
    A1 = data_cleaned.loc[i][index_A1.values]
    A1TMX = data_cleaned.loc[i][index_A1pbbrTMX].values
    p_values_A1A1TMX.append(sts.ttest_ind(A1, A1TMX, equal_var=False).pvalue)

## An example issue with multiple testing correction method: Bonferroni's correction

With prior knowledge, we expected that there would be at least some changes in chemoreceptors or other proteins with chemotaxis functions in our mutants. Here, we show that Bonferroni correction is very **conservative** multiple testing correction method because it eliminates all of the p < 0.05 elements to reduce type I errors. However, applying Bonferroni corrections also increase the rate of **false negative**. There are other methods of multiple testing correction method (https://www.statsmodels.org/stable/generated/statsmodels.stats.multitest.multipletests.html) that one could try.

We decided to not apply a correction methods. However, we are aware that the analysis on **trial 2** might not be as statistically strong as analysis on **trial 1**.

In [58]:
# From the statsmodels.stats package, we can perform the Bonferroni correction method using the 
# multitest module.

qvs_A1vssp7 = multitest.multipletests(p_values_A1sp7 ,method='fdr_bh', alpha=0.05)
qvs_A1TMXvssp7 = multitest.multipletests(p_values_A1TMXsp7 ,method='fdr_bh', alpha=0.05)
qvs_A1vsA1TMX = multitest.multipletests(p_values_A1A1TMX ,method='fdr_bh', alpha=0.05)

data_cleaned['p_correct_A1vssp7'] = qvs_A1vssp7[1]
data_cleaned['p_correct_A1TMXvssp7'] = qvs_A1TMXvssp7[1]
data_cleaned['p_correct_A1vsA1TMX'] = qvs_A1vsA1TMX[1]

#Now, we have to separate the proteins that has p<0.05 for at least one comparison study
data_DEPs = data_cleaned[(data_cleaned['p_correct_A1vssp7'] < 0.05) | (data_cleaned['p_correct_A1TMXvssp7'] < 0.05) | (data_cleaned['p_correct_A1vsA1TMX'] < 0.05)]
data_DEPs.head()

,Unnamed: 0,CheA1_pBBR_TMX_Rep01,CheA1_pBBR_TMX_Rep02,CheA1_pBBR_TMX_Rep03,CheA1_Rep01,CheA1_Rep02,CheA1_Rep03,sp7_Rep01,sp7_Rep02,sp7_Rep03,Accession,Gene function,p_correct_A1vssp7,p_correct_A1TMXvssp7,p_correct_A1vsA1TMX


In [59]:
data_cleaned['p_correct_A1vssp7'] = p_values_A1sp7
data_cleaned['p_correct_A1TMXvssp7'] = p_values_A1TMXsp7
data_cleaned['p_correct_A1vsA1TMX'] = p_values_A1A1TMX

#Now, we have to separate the proteins that has p<0.05 for at least one comparison study
data_DEPs = data_cleaned[(data_cleaned['p_correct_A1vssp7'] < 0.05) | (data_cleaned['p_correct_A1TMXvssp7'] < 0.05) | (data_cleaned['p_correct_A1vsA1TMX'] < 0.05)].reset_index(drop=True)
data_DEPs.head()

,Unnamed: 0,CheA1_pBBR_TMX_Rep01,CheA1_pBBR_TMX_Rep02,CheA1_pBBR_TMX_Rep03,CheA1_Rep01,CheA1_Rep02,CheA1_Rep03,sp7_Rep01,sp7_Rep02,sp7_Rep03,Accession,Gene function,p_correct_A1vssp7,p_correct_A1TMXvssp7,p_correct_A1vsA1TMX
0,16,29.682458,28.780629,28.815896,30.209004,29.422380,30.518898,30.092997,30.733791,30.465057,A0A060DBW2,Acyl carrier protein (ACP),0.381844,0.025316,0.095875
1,24,28.301680,28.678478,28.224968,28.549411,28.712650,28.584363,29.113264,28.856076,29.399934,A0A060DDF9,50S ribosomal protein L21,0.072534,0.027154,0.263228
2,25,29.280890,29.427257,29.482952,28.818920,28.377499,28.456413,29.451072,29.174875,29.204591,A0A060DDG3,DUF465 domain-containing protein,0.015558,0.328675,0.013473
3,27,32.513148,32.672803,32.651796,32.082652,32.199736,32.339821,32.369120,32.736787,32.679752,A0A060DDI4,Cold-shock protein (Putative cold-shock DNA-bi...,0.055711,0.898886,0.014390
4,36,26.588165,26.235722,26.671305,28.034110,26.608943,27.684494,28.101245,27.734091,27.279743,A0A060DEF4,Phosphatidylserine decarboxylase proenzyme (EC...,0.628135,0.019331,0.149397


In [60]:
mean_sp7 = np.mean(data_DEPs[index_sp7], axis=1)
mean_A1 = np.mean(data_DEPs[index_A1], axis=1)
mean_A1TMX = np.mean(data_DEPs[index_A1pbbrTMX], axis=1)

#This is a different formula because the abundances are already in log2 values
data_DEPs['log2_A1/sp7'] = mean_A1 - mean_sp7
data_DEPs['log2_A1TMX/sp7'] = mean_A1TMX - mean_sp7
data_DEPs['log2_A1/A1TMX'] = mean_A1 - mean_A1TMX

Below is our differentially expressed protein dataset for **trial 2**.

In [61]:
data_DEPs = data_DEPs[(abs(data_DEPs['log2_A1/sp7']) > 1) | (abs(data_DEPs['log2_A1TMX/sp7']) > 1) | (abs(data_DEPs['log2_A1/A1TMX']) > 1)]
data_DEPs = data_DEPs.reset_index(drop=True)[data_DEPs.columns[1:len(data_DEPs.columns)]]
data_DEPs.head()

,CheA1_pBBR_TMX_Rep01,CheA1_pBBR_TMX_Rep02,CheA1_pBBR_TMX_Rep03,CheA1_Rep01,CheA1_Rep02,CheA1_Rep03,sp7_Rep01,sp7_Rep02,sp7_Rep03,Accession,Gene function,p_correct_A1vssp7,p_correct_A1TMXvssp7,p_correct_A1vsA1TMX,log2_A1/sp7,log2_A1TMX/sp7,log2_A1/A1TMX
0,29.682458,28.780629,28.815896,30.209004,29.422380,30.518898,30.092997,30.733791,30.465057,A0A060DBW2,Acyl carrier protein (ACP),0.381844,0.025316,0.095875,-0.380521,-1.337621,0.957100
1,26.588165,26.235722,26.671305,28.034110,26.608943,27.684494,28.101245,27.734091,27.279743,A0A060DEF4,Phosphatidylserine decarboxylase proenzyme (EC...,0.628135,0.019331,0.149397,-0.262511,-1.206629,0.944118
2,27.146370,27.508063,27.661259,28.390654,27.960768,29.168877,28.979082,29.503765,30.593822,A0A060DFR0,MucR family transcriptional regulator (Transcr...,0.121887,0.032033,0.077444,-1.185457,-2.253659,1.068202
3,26.829908,24.609909,24.092496,24.461508,23.922084,26.188228,28.217229,28.071609,27.769660,A0A060DG81,Urease subunit gamma (EC 3.5.1.5) (Urea amidoh...,0.039455,0.073926,0.782729,-3.162226,-2.842062,-0.320164
4,31.203365,31.392614,30.631203,30.528273,29.868579,30.030057,30.056165,29.674910,29.384763,A0A060DGY9,Protein HflC,0.190911,0.010952,0.037890,0.437023,1.370448,-0.933425
